In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

import wandb
wandb.login()

random.seed()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cs20b013 (cs20b013-bersilin). Use `wandb login --relogin` to force relogin


cuda


In [2]:
# Language Model
SOS_token = 0
EOS_token = 1

class Language:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {SOS_token: "<", EOS_token: ">"}
        self.n_chars = 2  # Count SOS and EOS

    def addWord(self, word):
        for char in word:
            self.addChar(char)

    def addChar(self, char):
        if char not in self.word2index:
            self.word2index[char] = self.n_chars
            self.word2count[char] = 1
            self.index2word[self.n_chars] = char
            self.n_chars += 1
        else:
            self.word2count[char] += 1

In [3]:
def get_data(lang: str, type: str) -> list[list[str]]:
    """
    Returns: 'pairs': list of [input_word, target_word] pairs
    """
    path = "./aksharantar_sampled/{}/{}_{}.csv".format(lang, lang, type)
    df = pd.read_csv(path, header=None)
    pairs = df.values.tolist()
    return pairs

In [4]:
def get_languages(lang: str):
    """
    Returns 
    1. input_lang: input language - English
    2. output_lang: output language - Given language
    3. pairs: list of [input_word, target_word] pairs
    """
    input_lang = Language('eng')
    output_lang = Language(lang)
    pairs = get_data(lang, "train")
    for pair in pairs:
        input_lang.addWord(pair[0])
        output_lang.addWord(pair[1])
    return input_lang, output_lang, pairs

In [5]:
def get_cell(cell_type: str):
    if cell_type == "LSTM":
        return nn.LSTM
    elif cell_type == "GRU":
        return nn.GRU
    elif cell_type == "RNN":
        return nn.RNN
    else:
        raise Exception("Invalid cell type")
    
def get_optimizer(optimizer: str):
    if optimizer == "SGD":
        return optim.SGD
    elif optimizer == "ADAM":
        return optim.Adam
    else:
        raise Exception("Invalid optimizer")

In [6]:
class Encoder(nn.Module):
    def __init__(self,
                 in_sz: int,
                 embed_sz: int,
                 hidden_sz: int,
                 cell_type: str,
                 n_layers: int,
                 dropout: float):
        
        super(Encoder, self).__init__()
        self.hidden_sz = hidden_sz
        self.n_layers = n_layers
        self.dropout = dropout
        self.cell_type = cell_type
        self.embedding = nn.Embedding(in_sz, embed_sz)

        self.rnn = get_cell(cell_type)(input_size = embed_sz,
                                       hidden_size = hidden_sz,
                                       num_layers = n_layers,
                                       dropout = dropout)
        
    def forward(self, input, hidden, cell):
        embedded = self.embedding(input).view(1, 1, -1)

        if(self.cell_type == "LSTM"):
            output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        else:
            output, hidden = self.rnn(embedded, hidden)
            
        return output, hidden, cell
    
    def initHidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_sz, device=device)

In [7]:
class AttentionDecoder(nn.Module):
    def __init__(self,
                 out_sz: int,
                 embed_sz: int,
                 hidden_sz: int,
                 cell_type: str,
                 n_layers: int,
                 dropout: float):

        super(AttentionDecoder, self).__init__()
        self.hidden_sz = hidden_sz
        self.n_layers = n_layers
        self.dropout = dropout
        self.cell_type = cell_type
        self.embedding = nn.Embedding(out_sz, embed_sz)

        self.attn = nn.Linear(hidden_sz + embed_sz, 50)
        self.attn_combine = nn.Linear(hidden_sz + embed_sz, hidden_sz)

        self.rnn = get_cell(cell_type)(input_size = hidden_sz,
                                       hidden_size = hidden_sz,
                                       num_layers = n_layers,
                                       dropout = dropout)
        
        self.out = nn.Linear(hidden_sz, out_sz)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell, encoder_outputs):
        embedding = self.embedding(input).view(1, 1, -1)

        attn_weights = F.softmax(self.attn(torch.cat((embedding[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedding[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        if(self.cell_type == "LSTM"):
            output, (hidden, cell) = self.rnn(output, (hidden, cell))
        else:
            output, hidden = self.rnn(output, hidden)

        output = self.softmax(self.out(output[0]))
        return output, hidden, cell, attn_weights
    
    def initHidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_sz, device=device)

In [8]:
def indexesFromWord(lang:Language, word:str):
    return [lang.word2index[char] for char in word]

def tensorFromWord(lang:Language, word:str):
    indexes = indexesFromWord(lang, word)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(input_lang:Language, output_lang:Language, pair:list[str]):
    input_tensor = tensorFromWord(input_lang, pair[0])
    target_tensor = tensorFromWord(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [9]:
def params_definition(): 
    """
    params:

        embed_size : size of embedding (input and output) (8, 16, 32, 64)
        hidden_size : size of hidden layer (64, 128, 256, 512)
        cell_type : type of cell (LSTM, GRU, RNN)
        num_layers : number of layers in encoder (1, 2, 3)
        dropout : dropout probability
        learning_rate : learning rate
        teacher_forcing_ratio : teacher forcing ratio (0.5 fixed for now)
        optimizer : optimizer (SGD, Adam)
        max_length : maximum length of input word (50 fixed for now)

    """
    pass

In [10]:
PRINT_EVERY = 5000
PLOT_EVERY = 100

In [11]:
class Translator:
    def __init__(self, lang: str, params: dict):
        self.lang = lang
        self.input_lang, self.output_lang, self.pairs = get_languages(self.lang)
        self.input_size = self.input_lang.n_chars
        self.output_size = self.output_lang.n_chars

        self.training_pairs = [tensorsFromPair(self.input_lang, self.output_lang, pair) for pair in self.pairs]

        self.encoder = Encoder(in_sz = self.input_size,
                             embed_sz = params["embed_size"],
                             hidden_sz = params["hidden_size"],
                             cell_type = params["cell_type"],
                             n_layers = params["num_layers"],
                             dropout = params["dropout"]).to(device)
        
        self.decoder = AttentionDecoder(out_sz = self.output_size,
                             embed_sz = params["embed_size"],
                             hidden_sz = params["hidden_size"],
                             cell_type = params["cell_type"],
                             n_layers = params["num_layers"],
                             dropout = params["dropout"]).to(device)

        self.encoder_optimizer = get_optimizer(params["optimizer"])(self.encoder.parameters(), lr=params["learning_rate"], weight_decay=params["weight_decay"])
        self.decoder_optimizer = get_optimizer(params["optimizer"])(self.decoder.parameters(), lr=params["learning_rate"], weight_decay=params["weight_decay"])
        
        self.criterion = nn.NLLLoss()

        self.teacher_forcing_ratio = params["teacher_forcing_ratio"]
        self.max_length = params["max_length"]

    def train_single(self, input_tensor, target_tensor):
        encoder_hidden = self.encoder.initHidden()
        encoder_cell = self.encoder.initHidden()

        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()

        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)

        encoder_outputs = torch.zeros(self.max_length, self.encoder.hidden_sz, device=device)

        loss = 0

        for ei in range(input_length):
            encoder_output, encoder_hidden, encoder_cell = self.encoder(input_tensor[ei], encoder_hidden, encoder_cell)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden, decoder_cell = encoder_hidden, encoder_cell

        use_teacher_forcing = True if random.random() < self.teacher_forcing_ratio else False

        if use_teacher_forcing:
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_cell, decoder_attention = self.decoder(decoder_input, decoder_hidden, decoder_cell, encoder_outputs)
                loss += self.criterion(decoder_output, target_tensor[di])

                decoder_input = target_tensor[di]
        else:
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_cell, decoder_attention = self.decoder(decoder_input, decoder_hidden, decoder_cell, encoder_outputs)
                loss += self.criterion(decoder_output, target_tensor[di])

                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()
                if decoder_input.item() == EOS_token:
                    break

        loss.backward()
        self.encoder_optimizer.step()
        self.decoder_optimizer.step()

        return loss.item() / target_length
    
    def train(self, iters=-1):
        start_time = time.time()
        plot_losses = []
        print_loss_total = 0
        plot_loss_total = 0

        random.shuffle(self.training_pairs)
        iters = len(self.training_pairs) if iters == -1 else iters

        for iter in range(1, iters):
            training_pair = self.training_pairs[iter - 1]
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = self.train_single(input_tensor, target_tensor)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % PRINT_EVERY == 0:
                print_loss_avg = print_loss_total / PRINT_EVERY
                print_loss_total = 0
                current_time = time.time()
                print("Loss: {:.4f} | Iterations: {} | Time: {:.3f}".format(print_loss_avg, iter, current_time - start_time))

            if iter % PLOT_EVERY == 0:
                plot_loss_avg = plot_loss_total / PLOT_EVERY
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0
            
        return plot_losses
    
    def evaluate(self, word):
        with torch.no_grad():
            input_tensor = tensorFromWord(self.input_lang, word)
            input_length = input_tensor.size()[0]
            encoder_hidden = self.encoder.initHidden()
            encoder_cell = self.encoder.initHidden()

            encoder_outputs = torch.zeros(self.max_length, self.encoder.hidden_sz, device=device)

            for ei in range(input_length):
                encoder_output, encoder_hidden, encoder_cell = self.encoder(input_tensor[ei], encoder_hidden, encoder_cell)
                encoder_outputs[ei] += encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden, decoder_cell = encoder_hidden, encoder_cell

            decoded_chars = ""
            decoder_attentions = torch.zeros(self.max_length, self.max_length)

            for di in range(self.max_length):
                decoder_output, decoder_hidden, decoder_cell, decoder_attention = self.decoder(decoder_input, decoder_hidden, decoder_cell, encoder_outputs)
                decoder_attentions[di] = decoder_attention.data
                topv, topi = decoder_output.topk(1)
                
                if topi.item() == EOS_token:
                    break
                else:
                    decoded_chars += self.output_lang.index2word[topi.item()]

                decoder_input = topi.squeeze().detach()

            return decoded_chars, decoder_attentions[:di + 1]
        
    def test_validate(self, type:str):
        pairs = get_data(self.lang, type)
        accuracy = 0
        for pair in pairs:
            output, _ = self.evaluate(pair[0])
            if output == pair[1]:
                accuracy += 1
        return accuracy / len(pairs)

In [12]:
def plot_losses(plot_losses, title: str):
    # return plot of losses
    x_labels = [i * PLOT_EVERY for i in range(1, len(plot_losses) + 1)]
    plt.plot(x_labels, plot_losses, color="blue")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.title(title)
    plt.show()

In [13]:
sweep_configuration = {
    "method": "bayes",
    "metric": {
        "name": "validation_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "embed_size": {
            "values": [8, 16, 32]
        },
        "hidden_size": {
            "values": [128, 256, 512]
        },
        "cell_type": {
            "values": ["GRU", "LSTM", "RNN"]
        },
        "num_layers": {
            "values": [1, 2, 3]
        },
        "dropout": {
            "values": [0, 0.1, 0.2]
        },
        "learning_rate": {
            "values": [0.001, 0.005]
        },
        "optimizer": {
            "values": ["SGD", "ADAM"]
        },
        "teacher_forcing_ratio": {
            'value': 0.5
        },
        "max_length": {
            'value': 50
        },
        "weight_decay": {
            "values": [0, 1e-1, 1e-3, 1e-5]
        }
    }
}

In [14]:
count = 0

def train_sweep():
    global count
    count += 1

    run = wandb.init()
    config = wandb.config
    run.name = "embed_size: {} | hidden_size: {} | cell_type: {} | num_layers: {} | dropout: {} | learning_rate: {} | optimizer: {} | teacher_forcing_ratio: {} | max_length: {} | weight_decay: {}".format(config.embed_size, config.hidden_size, config.cell_type, config.num_layers, config.dropout, config.learning_rate, config.optimizer, config.teacher_forcing_ratio, config.max_length, config.weight_decay)

    model = Translator('tam', config)

    epochs = 10

    for epoch in range(epochs):
        print("Epoch: {}".format(epoch + 1))
        plot_losses = model.train()

        # take average of plot losses as training loss
        training_loss = sum(plot_losses) / len(plot_losses)
        
        training_accuracy = model.test_validate('train')
        print("Training Accuracy: {:.4f}".format(training_accuracy))

        validation_accuracy = model.test_validate('valid')
        print("Validation Accuracy: {:.4f}".format(validation_accuracy))

        wandb.log({
            "epoch": epoch + 1,
            "training_loss": training_loss,
            "training_accuracy": training_accuracy,
            "validation_accuracy": validation_accuracy
        })

        if epoch > 0:
            if validation_accuracy < 0.0001:
                break

    test_accuracy = model.test_validate('test')
    print("Test Accuracy: {:.4f}".format(test_accuracy))

    wandb.log({
        "test_accuracy": test_accuracy
    })

    run.finish()

In [15]:
sweep_id = wandb.sweep(sweep_configuration, project="Test2")
wandb.agent(sweep_id, train_sweep, count=10)

Create sweep with ID: w613tuxy
Sweep URL: https://wandb.ai/cs20b013-bersilin/Test2/sweeps/w613tuxy


wandb: Agent Starting Run: 3vzxdlwf with config:
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.1
wandb: 	embed_size: 32
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.005
wandb: 	max_length: 50
wandb: 	num_layers: 2
wandb: 	optimizer: SGD
wandb: 	teacher_forcing_ratio: 0.5
wandb: 	weight_decay: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch: 1
Loss: 2.9721 | Iterations: 5000 | Time: 58.193
Loss: 2.8979 | Iterations: 10000 | Time: 130.161
Loss: 2.8853 | Iterations: 15000 | Time: 214.936
Loss: 2.8362 | Iterations: 20000 | Time: 297.428
Loss: 2.8325 | Iterations: 25000 | Time: 381.752
Loss: 2.8172 | Iterations: 30000 | Time: 467.418
Loss: 2.8094 | Iterations: 35000 | Time: 551.827
Loss: 2.8117 | Iterations: 40000 | Time: 637.342
Loss: 2.8111 | Iterations: 45000 | Time: 721.459
Loss: 2.8023 | Iterations: 50000 | Time: 803.986
Training Accuracy: 0.0000
Validation Accuracy: 0.0000
Epoch: 2
Loss: 2.8088 | Iterations: 5000 | Time: 60.287
Loss: 2.8034 | Iterations: 10000 | Time: 119.788
Loss: 2.8066 | Iterations: 15000 | Time: 194.650
Loss: 2.8106 | Iterations: 20000 | Time: 271.471
Loss: 2.8147 | Iterations: 25000 | Time: 348.245
Loss: 2.8052 | Iterations: 30000 | Time: 425.318
Loss: 2.8012 | Iterations: 35000 | Time: 502.160
Loss: 2.8096 | Iterations: 40000 | Time: 578.641
Loss: 2.8068 | Iterations: 45000 | Time: 654.992
L

epoch,▁█
test_accuracy,▁
training_accuracy,▁▁
training_loss,█▁
validation_accuracy,▁▁
epoch,2
test_accuracy,0.0
training_accuracy,0.0
training_loss,2.80671
validation_accuracy,0.0


wandb: Agent Starting Run: rx8yaqdy with config:
wandb: 	cell_type: RNN
wandb: 	dropout: 0.1
wandb: 	embed_size: 16
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.005
wandb: 	max_length: 50
wandb: 	num_layers: 3
wandb: 	optimizer: ADAM
wandb: 	teacher_forcing_ratio: 0.5
wandb: 	weight_decay: 1e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

Epoch: 1
Loss: 2.6482 | Iterations: 5000 | Time: 116.914
Loss: 2.4904 | Iterations: 10000 | Time: 220.763
Loss: 1.9186 | Iterations: 15000 | Time: 289.225
Loss: 1.0822 | Iterations: 20000 | Time: 359.174
Loss: 0.7007 | Iterations: 25000 | Time: 428.109
Loss: 0.5542 | Iterations: 30000 | Time: 498.700
Loss: 0.4607 | Iterations: 35000 | Time: 568.792
Loss: 0.4289 | Iterations: 40000 | Time: 638.527
Loss: 0.3887 | Iterations: 45000 | Time: 708.300
Loss: 0.3476 | Iterations: 50000 | Time: 778.356
Training Accuracy: 0.5652
Validation Accuracy: 0.4688
Epoch: 2
Loss: 0.3131 | Iterations: 5000 | Time: 120.924
Loss: 0.3339 | Iterations: 10000 | Time: 243.117
Loss: 0.3233 | Iterations: 15000 | Time: 365.319
Loss: 0.2932 | Iterations: 20000 | Time: 486.285
Loss: 0.3130 | Iterations: 25000 | Time: 604.011
Loss: 0.2921 | Iterations: 30000 | Time: 699.236
Loss: 0.2814 | Iterations: 35000 | Time: 766.116
Loss: 0.2865 | Iterations: 40000 | Time: 831.811
Loss: 0.2899 | Iterations: 45000 | Time: 897.768

epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
training_accuracy,▁▄▅▄▇▇▆▇█▇
training_loss,█▂▂▁▁▁▁▁▁▁
validation_accuracy,▁▄▄▄▇▇▆▅█▇
epoch,10
test_accuracy,0.45435
training_accuracy,0.73631
training_loss,0.16528
validation_accuracy,0.5647


wandb: Agent Starting Run: pyqg6bau with config:
wandb: 	cell_type: RNN
wandb: 	dropout: 0
wandb: 	embed_size: 32
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.001
wandb: 	max_length: 50
wandb: 	num_layers: 2
wandb: 	optimizer: SGD
wandb: 	teacher_forcing_ratio: 0.5
wandb: 	weight_decay: 0.001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch: 1
Loss: 2.6608 | Iterations: 5000 | Time: 57.448
Loss: 2.4295 | Iterations: 10000 | Time: 120.917
Loss: 1.7206 | Iterations: 15000 | Time: 185.008
Loss: 1.0218 | Iterations: 20000 | Time: 249.919
Loss: 0.6799 | Iterations: 25000 | Time: 314.668
Loss: 0.5113 | Iterations: 30000 | Time: 379.496
Loss: 0.4293 | Iterations: 35000 | Time: 444.791
Loss: 0.4044 | Iterations: 40000 | Time: 510.343
Loss: 0.3825 | Iterations: 45000 | Time: 575.106
Loss: 0.3674 | Iterations: 50000 | Time: 640.684
Training Accuracy: 0.5879
Validation Accuracy: 0.4856
Epoch: 2
Loss: 0.3235 | Iterations: 5000 | Time: 57.559
Loss: 0.3243 | Iterations: 10000 | Time: 125.127
Loss: 0.3238 | Iterations: 15000 | Time: 208.130
Loss: 0.2974 | Iterations: 20000 | Time: 291.232
Loss: 0.2935 | Iterations: 25000 | Time: 374.142
Loss: 0.2772 | Iterations: 30000 | Time: 456.972
Loss: 0.2955 | Iterations: 35000 | Time: 540.205
Loss: 0.2870 | Iterations: 40000 | Time: 623.144
Loss: 0.2500 | Iterations: 45000 | Time: 706.038
L

wandb: Ctrl + C detected. Stopping sweep.
